# Keywords extraction using GPT 4


In this notebook we will demonstrate a method for enabling GPT-3 to answer questions using a library of text as a reference, by using document embeddings and retrieval.

In [1]:
import openai
import requests
from num2words import num2words
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import ssl
from nltk.stem import SnowballStemmer
import re


openai.api_type = "azure"
openai.api_key = "ec689a20ec5c48448d0604115bf3a97c"
openai.api_base = "https://alnairgpt4japan.openai.azure.com/"
openai.api_version = "2023-07-01-preview" 

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 

r = requests.get(url, headers={"api-key": openai.api_key})

print(r.text)
COMPLETION='linecompletion' #This will correspond to the custom name you chose for your deployment when you deployed a model.
EMBEDDING_MODEL ='lineembeddings'

ModuleNotFoundError: No module named 'num2words'

# 1) Pre-processing

In [15]:
# This dataset has already been split into sections, one row for each section of the document.
df = pd.read_excel('HWB_Tourisim_combined.xlsx')
df.head(5)


,Title,Heading,Content
0,"Health, Wellbeing, and Biotech Asset Brief",Confidentiality,"Health, Wellbeing, and Biotech Asset Brief Con..."
1,"Health, Wellbeing, and Biotech Asset Brief",Document Information and history,"Health, Wellbeing, and Biotech Asset Brief Doc..."
2,"Health, Wellbeing, and Biotech Asset Brief",Disclaimer,"Health, Wellbeing, and Biotech Asset Brief Dis..."
3,"Health, Wellbeing, and Biotech Asset Brief",Table of Content,"Health, Wellbeing, and Biotech Asset Brief Tab..."
4,"Health, Wellbeing, and Biotech Asset Brief",Introduction and Purpose,"Health, Wellbeing, and Biotech Asset Brief Int..."


In [16]:
df['Tokens'] = df['Content'].apply(lambda Content: len(word_tokenize(str(Content))))
df.head(20)

,Title,Heading,Content,Tokens
0,"Health, Wellbeing, and Biotech Asset Brief",Confidentiality,"Health, Wellbeing, and Biotech Asset Brief Con...",11
1,"Health, Wellbeing, and Biotech Asset Brief",Document Information and history,"Health, Wellbeing, and Biotech Asset Brief Doc...",97
2,"Health, Wellbeing, and Biotech Asset Brief",Disclaimer,"Health, Wellbeing, and Biotech Asset Brief Dis...",52
3,"Health, Wellbeing, and Biotech Asset Brief",Table of Content,"Health, Wellbeing, and Biotech Asset Brief Tab...",402
4,"Health, Wellbeing, and Biotech Asset Brief",Introduction and Purpose,"Health, Wellbeing, and Biotech Asset Brief Int...",163
5,"Health, Wellbeing, and Biotech Asset Brief",Overview of THE LINE Development Framework,"Health, Wellbeing, and Biotech Asset Brief Ov...",256
6,"Health, Wellbeing, and Biotech Asset Brief",Objective is for THE LINE Development Framew...,"Health, Wellbeing, and Biotech Asset Brief Ob...",97
7,"Health, Wellbeing, and Biotech Asset Brief",Vision and Strategy Brief (Tier 1) description,"Health, Wellbeing, and Biotech Asset Brief Vis...",147
8,"Health, Wellbeing, and Biotech Asset Brief",Batch Modules Development Brief (Tier 2) des...,"Health, Wellbeing, and Biotech Asset Brief Bat...",139
9,"Health, Wellbeing, and Biotech Asset Brief",Module Design Briefs (Tier 3) description,"Health, Wellbeing, and Biotech Asset Brief Mod...",125


In [17]:
def keywords_extraction(
    query: str,
    df: pd.DataFrame,
    show_prompt: bool = False
) -> str:
    messages = [
        {"role" : "system", "content":"""
         Your task is to extract the keywords from the 'Content' column - maximum 5 keywords(keywords combinations)
         You should not extract more than 5 keywords/keyword combinations
         Where there is no content from which you can extract keyword combinations response should be: "no keywords can be extracted"
         You should extract keywords that are the most representative from the provided text, example:
         Text: 
         3.1. Residential  Sector Vision   
         The vision for the residential sector is to provide an exceptional living environment for residents, so that the 
         world’s best talent chooses NEOM over other global talent hubs. NEOM aims to leverage the benefits of THE LINE ’s vertical urbanism to provide a world -leading residential offering in terms of quality and liveability at a 
         globally competitive price point. The sector should meet the affordability requirements of NEOM’s population, 
         creating the best place on earth to live.
         Keywords: 
         Residential Sector;
         Vertical urbanism;
         Affordability;
         Population
         
         You should exlude stop words from extracted keywords: 'line', 'design', 'neom', 'the', 'to', 'and','a','an','or', 'be', 'of', 'with', 'will', 'in', 'for', 'as','that', 'on', 'is', 'are', 'asset', 'assets', 'out'
         """},
        {"role": "user", "content": """
        Derive keywords from the content:
        The different residential products across THE LINE will provide the following provisional unit areas   
 
        Product  Ultra -Premium,  Bedrooms    5-8, GFA (m2) / unit  873, GLA (m2) / unit  591,  Average Min. Efficiency (NLA / GFA %)   68%  ,  Household Size  8.3  ,GFA per Capita 105 
        Product High Premium,  Bedrooms     2-6, GFA (m2) / unit  481, GLA (m2) / unit  340 ,  Average Min. Efficiency (NLA / GFA %)   71% ,  Household Size   5.4  ,GFA per Capita 89 
        Product Premium,  Bedrooms     1-5, GFA (m2) / unit  374, GLA (m2) / unit  267 ,  Average Min. Efficiency (NLA / GFA %)   72% ,  Household Size   4.6  ,GFA per Capita 78 
        Product High Market,  Bedrooms     1-5, GFA (m2) / unit  276 GLA (m2) / unit  205 ,  Average Min. Efficiency (NLA / GFA %)   75% ,  Household Size   4.0  ,GFA per Capita 65 
        Product Upper Mid- Market,  Bedrooms     0-5, GFA (m2) / unit  200, GLA (m2) / unit  148 ,  Average Min. Efficiency (NLA / GFA %)   75%  ,  Household Size  3.5  ,GFA per Capita 55 
        Product Mid-Market,  Bedrooms     0-5, GFA (m2) / unit  150, GLA (m2) / unit  112 ,  Average Min. Efficiency (NLA / GFA %)   75% ,  Household Size   3.4  ,GFA per Capita 45 
        Product Standard,  Bedrooms     0-5, GFA (m2) / unit  135, GLA (m2) / unit  101 ,  Average Min. Efficiency (NLA / GFA %)   75%  ,  Household Size  3.6  ,GFA per Capita 39 
        Product Entry,  Bedrooms     0-5, GFA (m2) / unit  103, GLA (m2) / unit  82 ,  Average Min. Efficiency (NLA / GFA %)   80% ,  Household Size   3.0  ,GFA per Capita 37 
        Product High Staff,  Bedrooms     0-4, GFA (m2) / unit  87, GLA (m2) / unit  70,  Average Min. Efficiency (NLA / GFA %)    80% ,  Household Size   2.6  ,GFA per Capita  35 
        Product Mid-Staff,  Bedrooms    0-4, GFA (m2) / unit  77, GLA (m2) / unit  66 ,  Average Min. Efficiency (NLA / GFA %)   85%,  Household Size    2.4  ,GFA per Capita 35 
        Product Entry Staff,  Bedrooms     4-12, GFA (m2) / unit  163, GLA (m2) / unit  138 ,  Average Min. Efficiency (NLA / GFA %)   85% ,  Household Size   7.4  ,GFA per Capita 23 
        Product High Student,  Bedrooms     0-4, GFA (m2) / unit  147, GLA (m2) / unit  110 ,  Average Min. Efficiency (NLA / GFA %)   75% ,  Household Size   2.0  ,GFA per Capita 74 
        Product Mid-Student,  Bedrooms     0-6, GFA (m2) / unit  108, GLA (m2) / unit  86,  Average Min. Efficiency (NLA / GFA %)    80%,  Household Size    3.1  ,GFA per Capita 41 
        Product Student,  Bedrooms     0-8, GFA (m2) / unit  107, GLA (m2) / unit  90 ,  Average Min. Efficiency (NLA / GFA %)   85%,  Household Size    4.6  ,GFA per Capita 27 
        Product Co-Living Mid ,  Bedrooms    1-6, GFA (m2) / unit  125, GLA (m2) / unit  99 ,  Average Min. Efficiency (NLA / GFA %)   80% ,  Household Size   3.3  ,GFA per Capita 42 
        Product Co-Living Entry,  Bedrooms     1-10 , GFA (m2) / unit 153, GLA (m2) / unit  130,  Average Min. Efficiency (NLA / GFA %)    85%,  Household Size    6.6  ,GFA per Capita 28 
        Product Senior Living Premium,  Bedrooms     1-3, GFA (m2) / unit  226, GLA (m2) / unit  162 ,  Average Min. Efficiency (NLA / GFA %)   72% ,  Household Size   1.6  ,GFA per Capita 135 
        Product Senior Living Upper Mid-Market ,  Bedrooms    1-3, GFA (m2) / unit  132, GLA (m2) / unit  97 ,  Average Min. Efficiency (NLA / GFA %)   75%,  Household Size    1.5  ,GFA per Capita 83 
        Product Senior Living Standard,  Bedrooms     1-3, GFA (m2) / unit  111, GLA (m2) / unit  83 ,  Average Min. Efficiency (NLA / GFA %)   75%,  Household Size    1.4  ,GFA per Capita 72 
        *Note  table data is in draft form and subject to change in line with revenue, affordability,  and population targets  
        This suite of 19 asset s offers  a ‘shopping list ’ of typologies to apply to each module in alignment with its key 
        characteristics e.g., anchor assets , Mirror LINE vs Upper Valley  etc. 
        Layouts should provide options for both regional  and international standards . 
        """},
        {"role": "assistant", "content": """
         provisional unit areas;
         GFA (m2) per unit;
         GLA (m2) per unit;
         GFA per Capita;
         """},
         {"role": "user", "content": """
        Derive keywords from the content:
        4. Residential Assets  
        Residential real estate will exist in all modules and will encompass both for sale and rental tenures with the 
        intention for both asset typologies to be interchangeable depending on demand.  
        Primary Homes are classified as units with at least one income earner  with a job within NEOM. Second Homes 
        are those without an income earner and/or is not their main residence and include   
            • Student  
            • Senior Living  
            • Short -Term Lets  
            • Ultra -Premium  
 
        """},
        {"role": "assistant", "content": """
         Primary Homes;
         Second Homes;
         Student;
         Senior Living
         """},
        {"role": "user", "content": """
        Derive keywords from the content:
        18. Sustainability 
         Development proposals should be designed in accordance with THE LINE’s commitment to environmental sustainability including in the areas of  
        ● Climate adaptation and resilience - some considerations include  
        ○ Dual purpose walls for vertical farming and closed loop water recycling 
        ○ Bioreceptive solution concrete that allows grass to grow on facades, lime concrete roof surfaces with high cooling potential. Wind towers and/or earth air tunnels. Trombe walls and solar 
        ○ chimneys and retro reflective paints 
        ○ Rotational hardware to support movement of vegetation to optimise growing conditions 
        ○ Design of interior and exterior needs to be augmented to allow for growing vegetation e.g 'daylighting' light irrigation systems 
        ● Low carbon construction - some considerations include  
        ○ Materials like Made of Air's plastic should replace any fossil fuel based plastics within the home. Interface carpet tiles will be carbon negative by 2040 
        ○ HempCrete, Cem-Free, Carbicrete and Carbon8 aggregate are all possible construction materials. Similarly, studies at University of Boulder show that algae can be used to produce carbon negative concrete 
        ● Zero carbon homes - some considerations include  
        ○ Air pollution absorbing plants placed inside of buildings 
        ○ Photocatalytic paints will be used to absorb pollution 
        ○ Air pollution alarms monitor levels, with data centres required to support connectivity and store information on pollution levels 
        ● Green measures to reduce heat - some considerations include  
        ○ On a Behavioural level, for instance, mimicking the techniques of passive ventilation and temperature regulation observed in termite mounds, as shown in the CH2 Building, Melbourne 
        ○ Energy capturing algae windows, venus flytrap inspired shades that respond naturally to external environments and 'Indus’, a tile-based, modular bioreactor wall system, based on the principle of bioremediation, that uses algae to sequester pollutants from water 
        ● Water use and reuse - some considerations include  
        ○ Redesigned balconies and roofs to efficiently capture rainwater and solar light 
        ● Certifications, for example  
        ○ LEED, BREEAM, EnergyStar
        """},
        {"role": "assistant", "content": """
         environmental sustainability;
         climate adaptation;
         Low carbon;
         Zero carbon;
         Water use
         """}
    ]        
    
    messages.append({"role" : "user", "content":query})
    response = openai.ChatCompletion.create(
        engine=COMPLETION,
        messages=messages,
        temperature=0,
        )
    return '\n' + response['choices'][0]['message']['content']

In [18]:
print(keywords_extraction("""Derive keywords from the content: 
6. Layout / Volume Metrics 
Unit Type, NLA Unit Size - sqm, Unit Mix Ratio, Unit Capacity,  Household Size (International | Regional | Weighted Average), Room Count
Studio, N/A, N/A, N/A, N/A| N/A | N/A, N/A
1 Bed, 80 - 110, 15%, 2, 1.6 | 1.6 | 1.6, 5.5
2 Bed, 140 - 190, 20%, 5, 2.8|  2.7 | 2.7, 9.5
3 Bed, 220 - 280, 25%, 7, 3.5 | 4.0 | 4.0, 12
4 bed, 350 - 450, 25%, 8, 4.1 | 4.8 | 4.4, 14.5
5 Bed, 500 - 600, 10%, 10, 5.1 | 6.0 | 5.5, 18.5
6 Bed, N/A, N/A, N/A, N/A N/A N/A, N/A
Penthouse, 700+, 5%, 10, 5.1 | 6.0 | 5.5, 20
                       """, df))



         Unit Type;
         NLA Unit Size;
         Unit Mix Ratio;
         Unit Capacity;
         Household Size


In [19]:
print(keywords_extraction("Derive keywords from the content: text", df))


no keywords can be extracted


In [20]:
print(keywords_extraction("""Derive keywords from the content: 
18. Sustainability 
 Development proposals should be designed in accordance with THE LINE’s commitment to environmental sustainability including in the areas of  
● Climate adaptation and resilience - some considerations include  
○ Dual purpose walls for vertical farming and closed loop water recycling 
○ Bioreceptive solution concrete that allows grass to grow on facades, lime concrete roof surfaces with high cooling potential. Wind towers and/or earth air tunnels. Trombe walls and solar 
○ chimneys and retro reflective paints 
○ Rotational hardware to support movement of vegetation to optimise growing conditions 
○ Design of interior and exterior needs to be augmented to allow for growing vegetation e.g 'daylighting' light irrigation systems 
● Low carbon construction - some considerations include  
○ Materials like Made of Air's plastic should replace any fossil fuel based plastics within the home. Interface carpet tiles will be carbon negative by 2040 
○ HempCrete, Cem-Free, Carbicrete and Carbon8 aggregate are all possible construction materials. Similarly, studies at University of Boulder show that algae can be used to produce carbon negative concrete 
● Zero carbon homes - some considerations include  
○ Air pollution absorbing plants placed inside of buildings 
○ Photocatalytic paints will be used to absorb pollution 
○ Air pollution alarms monitor levels, with data centres required to support connectivity and store information on pollution levels 
● Green measures to reduce heat - some considerations include  
○ On a Behavioural level, for instance, mimicking the techniques of passive ventilation and temperature regulation observed in termite mounds, as shown in the CH2 Building, Melbourne 
○ Energy capturing algae windows, venus flytrap inspired shades that respond naturally to external environments and 'Indus’, a tile-based, modular bioreactor wall system, based on the principle of bioremediation, that uses algae to sequester pollutants from water 
● Water use and reuse - some considerations include  
○ Redesigned balconies and roofs to efficiently capture rainwater and solar light 
● Certifications, for example  
○ LEED, BREEAM, EnergyStar
                       """, df))



         environmental sustainability;
         climate adaptation;
         Low carbon;
         Zero carbon;
         Water use



In [21]:
print(keywords_extraction("""Derive keywords from the content: 
9. Room types and Dimensions (Indicative) 
Master Bedroom, Minimum - Sqm, 13.0, Maximum - Sqm, 16.5
Bedroom,  Minimum - Sqm, 12.6, Maximum - Sqm, 13.3
Bathroom, Minimum - Sqm, 6.0, Maximum - Sqm, 7.2
Ensuite, Minimum - Sqm, 2.8, Maximum - Sqm, 4.4
Kitchen, Minimum - Sqm, 7.7, Maximum - Sqm, 14.6
Living Room, Minimum - Sqm, 16.3, Maximum - Sqm, 21.8
Storage Room, Minimum - Sqm, 0.6, Maximum - Sqm, 2.3
Office, Minimum - Sqm, 4.6, Maximum - Sqm, 9.3
Powder room, Minimum - Sqm, 1.0, Maximum - Sqm, 1.9
Utility Room, Minimum - Sqm, 6.1, Maximum - Sqm, 7.1
                       """, df))

KeyboardInterrupt: 

In [ ]:
content_str = str(df['Content'])
print(content_str)

In [22]:
import time

batch_size = 100
num_batches = len(df) // batch_size + (1 if len(df) % batch_size else 0)

for batch in range(num_batches):
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, len(df))
    
    for index in range(start_idx, end_idx):
        try:
            content_str = str(df.at[index, 'Content'])
            extracted_keywords = keywords_extraction(content_str, df)
            
            # Debug print
            print(f"Extracted from row {index}: {extracted_keywords}")
            
            keywords_str = ''.join(map(str, extracted_keywords))
            df.at[index, 'Heading'] += ' ' + keywords_str
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            
            # Save the current state of df to Excel
            file_name = "intermediate_output.xlsx"
            df.to_excel(file_name, engine='openpyxl', index=False)
            
            # Wait for 60 seconds
            time.sleep(60)

# Optionally, save the final state of df once all rows are processed
file_name = "HWB_keywords.xlsx"
df.to_excel(file_name, engine='openpyxl', index=False)


Extracted from row 0: 
no keywords can be extracted
Extracted from row 1: 
no keywords can be extracted
Extracted from row 2: 
no keywords can be extracted
Extracted from row 3: 

         Sector Asset Vision and Strategy;
         Quaternary & Community Hospitals;
         NEOM Biotechnology Park;
         Rehabilitation and Post-op Recovery Resort;
         Medical Wellness Resort

Extracted from row 4: 

         Health, Wellbeing, and Biotech Asset;
         Vision and Strategy Brief;
         Design principles;
         Sector strategies;
         Development Framework
Extracted from row 5: 

         THE LINE Development Framework;
         planning framework;
         design features;
         development approach;
         asset-level design principles
Extracted from row 6: 
Health, Wellbeing, and Biotech Asset;
Development Framework;
Design Stage 3;
NEOM Plan of Work;
Design evolution
Extracted from row 7: 

         Tier 1;
         strategic direction;
         concept maste

Extracted from row 58: 
Community Hospital;
Advanced Health Center;
Family Health Center;
NEOM Biotechnology Park
Extracted from row 59: 

         Retail assets;
         Commercial Real Estate Assets;
         Hospitality;
         Entertainment and Culture;
         Anchor Assets

Extracted from row 60: 

         Development Framework;
         design principles;
         Sector;
         module;
         Design Stage 3

Extracted from row 61: 

         Tourism Asset;
         Development Framework;
         Strategic direction;
         Concept masterplans;
         Design development

Extracted from row 62: 

         Tourism Asset;
         The LINE Development Framework;
         Batch Modules Development;
         Economic conditions;
         Overarching ecosystem

Extracted from row 63: 

         Tourism Asset;
         Module Design Briefs;
         Development Principles;
         Universal Module;
         Land uses

Extracted from row 64: 
Tourism Asset Brief;
Design S